In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [20]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [21]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 1s 485us/step - 

1052/1052 [==============================] - 0s 90us/step - loss: 21.7126 - val_loss: 26.4958
Epoch 66/1000
1052/1052 [==============================] - 0s 92us/step - loss: 21.9320 - val_loss: 26.1411
Epoch 67/1000
1052/1052 [==============================] - 0s 89us/step - loss: 22.5440 - val_loss: 25.6644
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.6607 - val_loss: 25.8389
Epoch 69/1000
1052/1052 [==============================] - 0s 91us/step - loss: 21.5857 - val_loss: 25.5477
Epoch 70/1000
1052/1052 [==============================] - 0s 96us/step - loss: 21.7111 - val_loss: 26.9015
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.9441 - val_loss: 27.4790
Epoch 72/1000
1052/1052 [==============================] - ETA: 0s - loss: 22.72 - 0s 86us/step - loss: 21.8520 - val_loss: 27.2168
Epoch 73/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.6825 - val_loss: 25.8909
Epoch 74/1000
1052

1052/1052 [==============================] - 0s 91us/step - loss: 21.4253 - val_loss: 26.3811
Epoch 141/1000
1052/1052 [==============================] - 0s 96us/step - loss: 21.6183 - val_loss: 25.9107
Epoch 142/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.3077 - val_loss: 24.9708
Epoch 143/1000
1052/1052 [==============================] - 0s 87us/step - loss: 21.1383 - val_loss: 25.5722
Epoch 144/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.1433 - val_loss: 28.0719
Epoch 145/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.6057 - val_loss: 25.4628
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 20.5926 - val_loss: 25.2125
Epoch 147/1000
1052/1052 [==============================] - 0s 91us/step - loss: 20.7283 - val_loss: 24.3924
Epoch 148/1000
1052/1052 [==============================] - 0s 89us/step - loss: 20.8032 - val_loss: 24.3128
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 90us/step - loss: 14.7134 - val_loss: 17.5741
Epoch 216/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.5678 - val_loss: 18.2298
Epoch 217/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.7401 - val_loss: 19.4454
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.1028 - val_loss: 16.4808
Epoch 219/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.7663 - val_loss: 20.7298
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.9431 - val_loss: 16.9210
Epoch 221/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.0195 - val_loss: 16.9503
Epoch 222/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.1613 - val_loss: 17.4071
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.3983 - val_loss: 17.6971
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 10.6072 - val_loss: 13.3669
Epoch 291/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.1618 - val_loss: 12.1901
Epoch 292/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.1894 - val_loss: 12.8353
Epoch 293/1000
1052/1052 [==============================] - 0s 83us/step - loss: 10.0313 - val_loss: 11.9468
Epoch 294/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.0668 - val_loss: 13.1120
Epoch 295/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.5541 - val_loss: 12.6360
Epoch 296/1000
1052/1052 [==============================] - 0s 84us/step - loss: 10.0971 - val_loss: 12.2707
Epoch 297/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.9403 - val_loss: 12.7971
Epoch 298/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.7066 - val_loss: 13.2847
Epoch 299/1000
1052/1052 [=========

...

1052/1052 [==============================] - 0s 91us/step - loss: 8.7025 - val_loss: 10.5626
Epoch 744/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.5300 - val_loss: 10.5586
Epoch 745/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9198 - val_loss: 10.4640
Epoch 746/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7155 - val_loss: 10.3637
Epoch 747/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8876 - val_loss: 11.3862
Epoch 748/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.0657 - val_loss: 10.8814
Epoch 749/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.6983 - val_loss: 10.5361
Epoch 750/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8289 - val_loss: 10.5193
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.8866 - val_loss: 11.0140
Epoch 752/1000
1052/1052 [=================

Epoch 819/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.0641 - val_loss: 10.5678
Epoch 820/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7556 - val_loss: 10.8177
Epoch 821/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.8810 - val_loss: 10.7610
Epoch 822/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9185 - val_loss: 11.3101
Epoch 823/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.5719 - val_loss: 10.5796
Epoch 824/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9067 - val_loss: 10.4681
Epoch 825/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.9615 - val_loss: 11.1061
Epoch 826/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.0102 - val_loss: 11.6883
Epoch 827/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.8409 - val_loss: 10.3841
Epoch 828/1000
1052/1052 [==

1052/1052 [==============================] - 0s 90us/step - loss: 9.1436 - val_loss: 12.1122
Epoch 895/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.1187 - val_loss: 10.7949
Epoch 896/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7415 - val_loss: 10.9564
Epoch 897/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8110 - val_loss: 11.6986
Epoch 898/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9090 - val_loss: 11.3493
Epoch 899/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.7616 - val_loss: 12.3338
Epoch 900/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9793 - val_loss: 10.7071
Epoch 901/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6278 - val_loss: 11.3630
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9190 - val_loss: 10.5193
Epoch 903/1000
1052/1052 [=================

Epoch 970/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5243 - val_loss: 10.6193
Epoch 971/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.5627 - val_loss: 10.4173
Epoch 972/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2062 - val_loss: 10.6715
Epoch 973/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7510 - val_loss: 10.3237
Epoch 974/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.4693 - val_loss: 12.5570
Epoch 975/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.5363 - val_loss: 10.9619
Epoch 976/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.6512 - val_loss: 10.9071
Epoch 977/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.7635 - val_loss: 10.5739
Epoch 978/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.5014 - val_loss: 10.1626
Epoch 979/1000
1052/1052 [==

[array([[-1.9627422 ,  3.720286  , -0.2045466 ,  3.4614522 ,  2.397332  ],
        [ 0.34341007, -0.3091803 , -0.25689313,  0.044903  , -0.19827989],
        [ 0.2883909 ,  0.57452184, -0.36653545,  0.04839338, -0.0276359 ],
        [ 0.05623429, -0.17904729,  0.0989675 ,  0.20756425, -0.1935187 ],
        [-0.20686434,  0.4317331 , -0.19647674,  0.4245821 ,  3.0578465 ]],
       dtype=float32),
 array([-1.365446  ,  4.573095  , -0.10281154,  4.3669934 ,  4.452761  ],
       dtype=float32),
 array([[ 0.18102863,  0.85815674, -0.6823412 , -1.1811129 , -0.01965414,
         -1.1369356 , -1.0263681 , -0.04920809,  0.7137829 , -1.0292051 ],
        [ 2.115924  ,  2.4788651 , -2.2932673 , -2.2789063 , -1.5644004 ,
         -1.9995311 , -2.0674767 , -1.8962002 ,  1.9377713 , -1.2670631 ],
        [ 0.703413  ,  0.7431646 , -0.92807215, -1.0678416 , -0.74138147,
         -0.92932737, -1.0334527 , -0.76752645,  1.2805858 , -0.9856542 ],
        [ 2.062931  ,  1.2860991 , -1.2365801 , -2.264700

In [22]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_adam_2nd.h5')

In [23]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, sgd, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 164us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 75us/step - loss: 19.9626 - val_loss: 25.2523
Epoch 68/1000
1052/1052 [==============================] - 0s 73us/step - loss: 19.9111 - val_loss: 25.2522
Epoch 69/1000
1052/1052 [==============================] - 0s 75us/step - loss: 19.8333 - val_loss: 25.2463
Epoch 70/1000
1052/1052 [==============================] - 0s 70us/step - loss: 19.8077 - val_loss: 25.3171
Epoch 71/1000
1052/1052 [==============================] - 0s 72us/step - loss: 19.7787 - val_loss: 25.2083
Epoch 72/1000
1052/1052 [==============================] - 0s 71us/step - loss: 19.7857 - val_loss: 25.2545
Epoch 73/1000
1052/1052 [==============================] - 0s 69us/step - loss: 19.7176 - val_loss: 25.2308
Epoch 74/1000
1052/1052 [==============================] - 0s 68us/step - loss: 19.6779 - val_loss: 25.2065
Epoch 75/1000
1052/1052 [==============================] - 0s 73us/step - loss: 19.6302 - val_loss: 25.1707
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 68us/step - loss: 18.4192 - val_loss: 23.8490
Epoch 143/1000
1052/1052 [==============================] - 0s 65us/step - loss: 18.4549 - val_loss: 23.8600
Epoch 144/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.4050 - val_loss: 23.8352
Epoch 145/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.3924 - val_loss: 23.8268
Epoch 146/1000
1052/1052 [==============================] - 0s 67us/step - loss: 18.3583 - val_loss: 23.8450
Epoch 147/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.3897 - val_loss: 23.8213
Epoch 148/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.3461 - val_loss: 23.8039
Epoch 149/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.3286 - val_loss: 23.7757
Epoch 150/1000
1052/1052 [==============================] - 0s 69us/step - loss: 18.3250 - val_loss: 23.7785
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 69us/step - loss: 17.6393 - val_loss: 22.8230
Epoch 218/1000
1052/1052 [==============================] - 0s 74us/step - loss: 17.6406 - val_loss: 22.8060
Epoch 219/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.6324 - val_loss: 22.7407
Epoch 220/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.6290 - val_loss: 22.7855
Epoch 221/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.5934 - val_loss: 22.7165
Epoch 222/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.5712 - val_loss: 22.7424
Epoch 223/1000
1052/1052 [==============================] - 0s 75us/step - loss: 17.5966 - val_loss: 22.6648
Epoch 224/1000
1052/1052 [==============================] - 0s 75us/step - loss: 17.5821 - val_loss: 22.6585
Epoch 225/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.5574 - val_loss: 22.6827
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 77us/step - loss: 16.5730 - val_loss: 21.1125
Epoch 293/1000
1052/1052 [==============================] - 0s 73us/step - loss: 16.5490 - val_loss: 21.1186
Epoch 294/1000
1052/1052 [==============================] - 0s 73us/step - loss: 16.5298 - val_loss: 21.0653
Epoch 295/1000
1052/1052 [==============================] - 0s 76us/step - loss: 16.5188 - val_loss: 21.0251
Epoch 296/1000
1052/1052 [==============================] - 0s 74us/step - loss: 16.4999 - val_loss: 21.0049
Epoch 297/1000
1052/1052 [==============================] - 0s 73us/step - loss: 16.4924 - val_loss: 20.9882
Epoch 298/1000
1052/1052 [==============================] - 0s 72us/step - loss: 16.4581 - val_loss: 20.9624
Epoch 299/1000
1052/1052 [==============================] - 0s 71us/step - loss: 16.4406 - val_loss: 20.9234
Epoch 300/1000
1052/1052 [==============================] - 0s 66us/step - loss: 16.4356 - val_loss: 20.8816
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 73us/step - loss: 11.8944 - val_loss: 14.9720
Epoch 743/1000
1052/1052 [==============================] - 0s 71us/step - loss: 11.8943 - val_loss: 15.0051
Epoch 744/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.8854 - val_loss: 14.9418
Epoch 745/1000
1052/1052 [==============================] - 0s 74us/step - loss: 11.8824 - val_loss: 14.9921
Epoch 746/1000
1052/1052 [==============================] - 0s 71us/step - loss: 11.8870 - val_loss: 14.9507
Epoch 747/1000
1052/1052 [==============================] - 0s 74us/step - loss: 11.8762 - val_loss: 14.9848
Epoch 748/1000
1052/1052 [==============================] - 0s 74us/step - loss: 11.8672 - val_loss: 14.9718
Epoch 749/1000
1052/1052 [==============================] - 0s 78us/step - loss: 11.8618 - val_loss: 14.9660
Epoch 750/1000
1052/1052 [==============================] - 0s 73us/step - loss: 11.8674 - val_loss: 14.9587
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 85us/step - loss: 11.6606 - val_loss: 14.6577
Epoch 818/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.6486 - val_loss: 14.7028
Epoch 819/1000
1052/1052 [==============================] - 0s 81us/step - loss: 11.6442 - val_loss: 14.6631
Epoch 820/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.6480 - val_loss: 14.6623
Epoch 821/1000
1052/1052 [==============================] - 0s 81us/step - loss: 11.6450 - val_loss: 14.6701
Epoch 822/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.6487 - val_loss: 14.6527
Epoch 823/1000
1052/1052 [==============================] - 0s 79us/step - loss: 11.6433 - val_loss: 14.6359
Epoch 824/1000
1052/1052 [==============================] - 0s 77us/step - loss: 11.6357 - val_loss: 14.6387
Epoch 825/1000
1052/1052 [==============================] - 0s 81us/step - loss: 11.6290 - val_loss: 14.6769
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 71us/step - loss: 11.4766 - val_loss: 14.4091
Epoch 893/1000
1052/1052 [==============================] - 0s 76us/step - loss: 11.4812 - val_loss: 14.4320
Epoch 894/1000
1052/1052 [==============================] - 0s 69us/step - loss: 11.4682 - val_loss: 14.4252
Epoch 895/1000
1052/1052 [==============================] - 0s 78us/step - loss: 11.4642 - val_loss: 14.3959
Epoch 896/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.4668 - val_loss: 14.4336
Epoch 897/1000
1052/1052 [==============================] - 0s 70us/step - loss: 11.4616 - val_loss: 14.4041
Epoch 898/1000
1052/1052 [==============================] - 0s 67us/step - loss: 11.4522 - val_loss: 14.3682
Epoch 899/1000
1052/1052 [==============================] - 0s 69us/step - loss: 11.4566 - val_loss: 14.3862
Epoch 900/1000
1052/1052 [==============================] - 0s 67us/step - loss: 11.4591 - val_loss: 14.4249
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 71us/step - loss: 11.3195 - val_loss: 14.2220
Epoch 968/1000
1052/1052 [==============================] - 0s 70us/step - loss: 11.3221 - val_loss: 14.2256
Epoch 969/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.3232 - val_loss: 14.2047
Epoch 970/1000
1052/1052 [==============================] - 0s 71us/step - loss: 11.3224 - val_loss: 14.2536
Epoch 971/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.3507 - val_loss: 14.2438
Epoch 972/1000
1052/1052 [==============================] - 0s 72us/step - loss: 11.3331 - val_loss: 14.1683
Epoch 973/1000
1052/1052 [==============================] - 0s 69us/step - loss: 11.3335 - val_loss: 14.1969
Epoch 974/1000
1052/1052 [==============================] - 0s 74us/step - loss: 11.3157 - val_loss: 14.2370
Epoch 975/1000
1052/1052 [==============================] - 0s 70us/step - loss: 11.3160 - val_loss: 14.2119
Epoch 976/1000
1052/1052 [========

[array([[-1.6749653 ,  0.13568047,  0.987561  , -0.24620768,  0.1797436 ],
        [-0.0511227 ,  0.21228284,  0.13723049, -0.11128951,  0.57686746],
        [-0.38297492,  0.04553554,  0.6447571 , -0.16911775,  1.2377775 ],
        [ 0.17827928,  0.08998913, -0.04293243,  0.31685758, -0.06332519],
        [-0.30634502,  0.10743946,  0.3026511 , -0.19925404, -0.01468669]],
       dtype=float32),
 array([-0.19855885, -1.8309008 ,  1.7523012 ,  1.8597533 ,  0.95310223],
       dtype=float32),
 array([[ 0.10663555, -0.1547233 ,  0.9346577 , -0.43066826,  0.64625335,
          0.09651201, -0.04987983,  0.04926179, -0.9887888 , -0.5506319 ],
        [-1.0193207 , -1.5007395 , -1.5694028 , -0.00820394, -0.99245936,
         -1.5969888 , -0.5979531 , -0.17724748,  1.8640554 ,  0.14391185],
        [ 0.75803477,  1.5500326 ,  1.3832507 , -0.14690562,  0.21260327,
          1.8946211 ,  1.1575102 , -0.30698153, -1.0721582 ,  0.19947913],
        [ 0.04396914,  0.612629  ,  1.735404  ,  0.123041

In [24]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sgd_2nd.h5')

In [25]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, RMSprop, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 181us/step - l

1052/1052 [==============================] - 0s 78us/step - loss: 23.0920 - val_loss: 28.0292
Epoch 68/1000
1052/1052 [==============================] - 0s 76us/step - loss: 23.7194 - val_loss: 26.8861
Epoch 69/1000
1052/1052 [==============================] - 0s 81us/step - loss: 23.1872 - val_loss: 25.2917
Epoch 70/1000
1052/1052 [==============================] - 0s 81us/step - loss: 23.0823 - val_loss: 29.0217
Epoch 71/1000
1052/1052 [==============================] - 0s 81us/step - loss: 23.2337 - val_loss: 27.6560
Epoch 72/1000
1052/1052 [==============================] - 0s 81us/step - loss: 22.9855 - val_loss: 26.9395
Epoch 73/1000
1052/1052 [==============================] - 0s 77us/step - loss: 23.4469 - val_loss: 27.0372
Epoch 74/1000
1052/1052 [==============================] - 0s 79us/step - loss: 22.8334 - val_loss: 29.1998
Epoch 75/1000
1052/1052 [==============================] - 0s 80us/step - loss: 23.5689 - val_loss: 26.9401
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 80us/step - loss: 16.2492 - val_loss: 17.8183
Epoch 143/1000
1052/1052 [==============================] - 0s 75us/step - loss: 15.6564 - val_loss: 19.3042
Epoch 144/1000
1052/1052 [==============================] - 0s 76us/step - loss: 16.2765 - val_loss: 18.0124
Epoch 145/1000
1052/1052 [==============================] - 0s 81us/step - loss: 16.6445 - val_loss: 17.6649
Epoch 146/1000
1052/1052 [==============================] - 0s 78us/step - loss: 15.7383 - val_loss: 19.7796
Epoch 147/1000
1052/1052 [==============================] - 0s 76us/step - loss: 15.9238 - val_loss: 17.0887
Epoch 148/1000
1052/1052 [==============================] - 0s 83us/step - loss: 15.8421 - val_loss: 17.8914
Epoch 149/1000
1052/1052 [==============================] - 0s 77us/step - loss: 15.7286 - val_loss: 18.1411
Epoch 150/1000
1052/1052 [==============================] - 0s 77us/step - loss: 15.8900 - val_loss: 17.3264
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 81us/step - loss: 13.0998 - val_loss: 15.9631
Epoch 218/1000
1052/1052 [==============================] - 0s 79us/step - loss: 13.3765 - val_loss: 14.9979
Epoch 219/1000
1052/1052 [==============================] - 0s 81us/step - loss: 13.1134 - val_loss: 14.3080
Epoch 220/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.8482 - val_loss: 16.8124
Epoch 221/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.0431 - val_loss: 16.4284
Epoch 222/1000
1052/1052 [==============================] - 0s 84us/step - loss: 13.1266 - val_loss: 15.4340
Epoch 223/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.7608 - val_loss: 18.2980
Epoch 224/1000
1052/1052 [==============================] - 0s 81us/step - loss: 12.3583 - val_loss: 15.9811
Epoch 225/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.1329 - val_loss: 19.7969
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 80us/step - loss: 11.0820 - val_loss: 12.2233
Epoch 293/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.8022 - val_loss: 12.4155
Epoch 294/1000
1052/1052 [==============================] - 0s 79us/step - loss: 10.9581 - val_loss: 12.5409
Epoch 295/1000
1052/1052 [==============================] - 0s 80us/step - loss: 11.1293 - val_loss: 14.9197
Epoch 296/1000
1052/1052 [==============================] - 0s 91us/step - loss: 11.3623 - val_loss: 12.5810
Epoch 297/1000
1052/1052 [==============================] - 0s 83us/step - loss: 10.9233 - val_loss: 12.6173
Epoch 298/1000
1052/1052 [==============================] - 0s 76us/step - loss: 11.2153 - val_loss: 14.3053
Epoch 299/1000
1052/1052 [==============================] - 0s 81us/step - loss: 11.7709 - val_loss: 13.1236
Epoch 300/1000
1052/1052 [==============================] - 0s 79us/step - loss: 10.4735 - val_loss: 12.9278
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 81us/step - loss: 8.9281 - val_loss: 12.9158
Epoch 748/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.5445 - val_loss: 9.7088
Epoch 749/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.2368 - val_loss: 10.7715
Epoch 750/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.0325 - val_loss: 10.1737
Epoch 751/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.7112 - val_loss: 10.5287
Epoch 752/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.4554 - val_loss: 10.1885
Epoch 753/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.0091 - val_loss: 10.1888
Epoch 754/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.1782 - val_loss: 11.6999
Epoch 755/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.0419 - val_loss: 10.4734
Epoch 756/1000
1052/1052 [==================

1052/1052 [==============================] - 0s 82us/step - loss: 9.1732 - val_loss: 9.7932
Epoch 824/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.5069 - val_loss: 9.9144
Epoch 825/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.4481 - val_loss: 10.7344
Epoch 826/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.2470 - val_loss: 11.3505
Epoch 827/1000
1052/1052 [==============================] - 0s 75us/step - loss: 9.1195 - val_loss: 10.8675
Epoch 828/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.1108 - val_loss: 9.9300
Epoch 829/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.3577 - val_loss: 11.7961
Epoch 830/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.3824 - val_loss: 12.4562
Epoch 831/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.3861 - val_loss: 10.4210
Epoch 832/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 80us/step - loss: 8.9654 - val_loss: 11.8301
Epoch 900/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.0119 - val_loss: 15.6884
Epoch 901/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.1378 - val_loss: 9.5108
Epoch 902/1000
1052/1052 [==============================] - 0s 76us/step - loss: 9.5677 - val_loss: 9.7005
Epoch 903/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.7915 - val_loss: 10.1109
Epoch 904/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2523 - val_loss: 9.3128
Epoch 905/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.3259 - val_loss: 14.2389
Epoch 906/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.9833 - val_loss: 13.1988
Epoch 907/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2991 - val_loss: 9.4324
Epoch 908/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 85us/step - loss: 8.8881 - val_loss: 12.7450
Epoch 976/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.3232 - val_loss: 9.7385
Epoch 977/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.4502 - val_loss: 9.8853
Epoch 978/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.1014 - val_loss: 10.7154
Epoch 979/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.3419 - val_loss: 11.7185
Epoch 980/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2038 - val_loss: 13.9335
Epoch 981/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.8544 - val_loss: 14.5448
Epoch 982/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.1853 - val_loss: 9.7879
Epoch 983/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.0099 - val_loss: 9.6723
Epoch 984/1000
1052/1052 [=====================

[array([[-4.1340494 , -0.5423752 , -1.9246769 , -3.0257561 , -0.23880716],
        [ 0.22393963, -0.89263916,  0.43099388,  0.07349765, -0.28461948],
        [-0.73538846, -1.183267  ,  0.39100978, -0.02767474, -0.3983067 ],
        [ 0.17476942,  0.21309479,  0.05278031,  0.03071608,  0.23384993],
        [-0.388193  , -0.02713201, -0.27230966, -2.476717  , -0.24319181]],
       dtype=float32),
 array([-5.0068893, -0.5993848, -1.2175783, -4.599301 ,  1.956449 ],
       dtype=float32),
 array([[-1.3649524 ,  1.6944485 , -1.4146601 , -2.295778  , -1.7116433 ,
         -2.0746205 ,  1.0422294 ,  1.3980467 ,  1.9370059 , -2.1361299 ],
        [-0.34763578,  0.3017152 ,  0.9409002 ,  0.75136447,  0.7333163 ,
          0.81774217, -0.22864446, -0.7503742 , -0.7624513 , -0.30675095],
        [ 0.06575379, -0.22350077,  0.89081544,  0.54283416,  0.2821009 ,
         -0.0905275 , -1.0448612 , -1.099719  , -0.880763  ,  1.0795995 ],
        [-1.2549328 ,  1.2254385 , -1.6343043 , -2.0503776 , -

In [26]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_rmsprop_2nd.h5')

In [27]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 586us/step - loss:

Epoch 69/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7234 - val_loss: 8.7452
Epoch 70/1000
274/274 [==============================] - 0s 92us/step - loss: 6.6470 - val_loss: 8.5066
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6858 - val_loss: 8.7578
Epoch 72/1000
274/274 [==============================] - 0s 106us/step - loss: 7.1143 - val_loss: 9.0477
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 6.9774 - val_loss: 8.9867
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7534 - val_loss: 8.6095
Epoch 75/1000
274/274 [==============================] - 0s 106us/step - loss: 6.9065 - val_loss: 8.8270
Epoch 76/1000
274/274 [==============================] - 0s 106us/step - loss: 6.7294 - val_loss: 8.9848
Epoch 77/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6952 - val_loss: 8.9580
Epoch 78/1000
274/274 [==============================] - 

Epoch 147/1000
274/274 [==============================] - 0s 104us/step - loss: 6.4955 - val_loss: 8.0429
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 6.4364 - val_loss: 8.3037
Epoch 149/1000
274/274 [==============================] - 0s 88us/step - loss: 6.3809 - val_loss: 7.9521
Epoch 150/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2636 - val_loss: 8.2835
Epoch 151/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2055 - val_loss: 7.9917
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4069 - val_loss: 8.1047
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1609 - val_loss: 7.8708
Epoch 154/1000
274/274 [==============================] - 0s 105us/step - loss: 6.1599 - val_loss: 8.0165
Epoch 155/1000
274/274 [==============================] - 0s 109us/step - loss: 6.2539 - val_loss: 8.2186
Epoch 156/1000
274/274 [==========================

274/274 [==============================] - 0s 102us/step - loss: 5.9358 - val_loss: 8.3693
Epoch 225/1000
274/274 [==============================] - 0s 94us/step - loss: 6.1695 - val_loss: 9.4821
Epoch 226/1000
274/274 [==============================] - 0s 91us/step - loss: 6.4401 - val_loss: 8.8950
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2864 - val_loss: 9.5369
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2472 - val_loss: 8.6305
Epoch 229/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3103 - val_loss: 8.8690
Epoch 230/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0689 - val_loss: 8.3283
Epoch 231/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0035 - val_loss: 8.8260
Epoch 232/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8993 - val_loss: 8.6846
Epoch 233/1000
274/274 [==============================] - 0s 105us

274/274 [==============================] - 0s 98us/step - loss: 5.6497 - val_loss: 9.0096
Epoch 302/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7660 - val_loss: 8.9470
Epoch 303/1000
274/274 [==============================] - 0s 107us/step - loss: 5.6320 - val_loss: 8.6387
Epoch 304/1000
274/274 [==============================] - 0s 110us/step - loss: 5.7268 - val_loss: 9.0878
Epoch 305/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7339 - val_loss: 9.2700
Epoch 306/1000
274/274 [==============================] - 0s 97us/step - loss: 5.8993 - val_loss: 8.5724
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7384 - val_loss: 8.9630
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6346 - val_loss: 8.8199
Epoch 309/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7410 - val_loss: 8.5651
Epoch 310/1000
274/274 [==============================] - 0s 102

...

274/274 [==============================] - 0s 98us/step - loss: 5.5851 - val_loss: 9.5621
Epoch 765/1000
274/274 [==============================] - 0s 105us/step - loss: 5.7585 - val_loss: 11.5815
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4959 - val_loss: 10.2567
Epoch 767/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3012 - val_loss: 10.0022
Epoch 768/1000
274/274 [==============================] - 0s 96us/step - loss: 5.4047 - val_loss: 9.8445
Epoch 769/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4091 - val_loss: 10.1243
Epoch 770/1000
274/274 [==============================] - 0s 104us/step - loss: 5.3924 - val_loss: 10.5695
Epoch 771/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2693 - val_loss: 9.8081
Epoch 772/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4975 - val_loss: 10.7874
Epoch 773/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 95us/step - loss: 5.4768 - val_loss: 11.4811
Epoch 842/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4605 - val_loss: 9.8250
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5786 - val_loss: 10.5931
Epoch 844/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3439 - val_loss: 10.1206
Epoch 845/1000
274/274 [==============================] - 0s 104us/step - loss: 5.2420 - val_loss: 10.5295
Epoch 846/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4249 - val_loss: 9.7780
Epoch 847/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2991 - val_loss: 10.3677
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3092 - val_loss: 10.0154
Epoch 849/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3063 - val_loss: 9.7456
Epoch 850/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 101us/step - loss: 5.2393 - val_loss: 9.8013
Epoch 919/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2539 - val_loss: 11.8156
Epoch 920/1000
274/274 [==============================] - 0s 104us/step - loss: 5.6309 - val_loss: 10.1462
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7144 - val_loss: 11.7137
Epoch 922/1000
274/274 [==============================] - 0s 113us/step - loss: 5.9472 - val_loss: 9.8926
Epoch 923/1000
274/274 [==============================] - 0s 113us/step - loss: 5.2776 - val_loss: 10.7821
Epoch 924/1000
274/274 [==============================] - 0s 109us/step - loss: 5.9214 - val_loss: 11.1033
Epoch 925/1000
274/274 [==============================] - 0s 105us/step - loss: 5.8023 - val_loss: 9.8330
Epoch 926/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9367 - val_loss: 11.7063
Epoch 927/1000
274/274 [==============================] 

274/274 [==============================] - 0s 104us/step - loss: 5.2888 - val_loss: 10.6488
Epoch 996/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2131 - val_loss: 10.4896
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3115 - val_loss: 10.2330
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2304 - val_loss: 10.3609
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2446 - val_loss: 10.4470
Epoch 1000/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3719 - val_loss: 9.9938
6.785681579072596


[array([[ 0.23527598,  0.18306047,  0.630229  ,  0.8335271 ,  0.16864413],
        [-0.65516126,  0.09108444, -0.5733426 , -3.076668  , -2.27919   ],
        [ 0.24807127, -0.601133  ,  0.22050197, -4.3621316 , -1.9919835 ],
        [ 0.8391864 , -0.05638067, -0.5408989 ,  1.3992935 ,  1.1614571 ],
        [ 0.08347924,  0.49276072, -1.1711944 , -1.2726746 , -0.8501271 ],
        [-1.0115194 , -0.4390346 ,  1.3667979 , -0.35400954, -0.50223213],
        [ 0.04465767, -0.03045928,  0.9625578 ,  1.9132023 ,  0.40727934]],
       dtype=float32),
 array([ 0.9730627 , -0.86702555,  2.3616784 ,  0.6365921 , -0.2114679 ],
       dtype=float32),
 array([[ 0.4725439 , -0.9842037 ,  0.63340497, -0.3650821 , -0.6513353 ,
          0.21806085,  0.94043946, -0.26007622, -0.2119163 , -0.8563147 ],
        [-0.8652151 ,  0.2767465 ,  0.07263525,  0.67462254,  1.1010232 ,
         -0.6880427 , -0.27892098,  0.4513003 ,  0.05095565,  0.9372603 ],
        [-0.5553656 , -0.14808664,  0.04055563,  0.08967

In [28]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_adam_2nd.h5')

In [29]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, sgd, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 509us/step - loss:

274/274 [==============================] - 0s 84us/step - loss: 416.5824 - val_loss: 437.9178
Epoch 68/1000
274/274 [==============================] - 0s 80us/step - loss: 412.3758 - val_loss: 433.4041
Epoch 69/1000
274/274 [==============================] - 0s 79us/step - loss: 408.1419 - val_loss: 428.8218
Epoch 70/1000
274/274 [==============================] - 0s 80us/step - loss: 403.8501 - val_loss: 424.1724
Epoch 71/1000
274/274 [==============================] - 0s 80us/step - loss: 399.4738 - val_loss: 419.4849
Epoch 72/1000
274/274 [==============================] - 0s 80us/step - loss: 395.0726 - val_loss: 414.7162
Epoch 73/1000
274/274 [==============================] - 0s 80us/step - loss: 390.6288 - val_loss: 409.8833
Epoch 74/1000
274/274 [==============================] - 0s 80us/step - loss: 386.1241 - val_loss: 404.9686
Epoch 75/1000
274/274 [==============================] - 0s 80us/step - loss: 381.5279 - val_loss: 400.0697
Epoch 76/1000
274/274 [===================

Epoch 143/1000
274/274 [==============================] - 0s 84us/step - loss: 56.3745 - val_loss: 60.8549
Epoch 144/1000
274/274 [==============================] - 0s 84us/step - loss: 54.1235 - val_loss: 58.6303
Epoch 145/1000
274/274 [==============================] - 0s 80us/step - loss: 52.1164 - val_loss: 56.4309
Epoch 146/1000
274/274 [==============================] - 0s 91us/step - loss: 50.1644 - val_loss: 54.3407
Epoch 147/1000
274/274 [==============================] - 0s 84us/step - loss: 48.3065 - val_loss: 52.3800
Epoch 148/1000
274/274 [==============================] - 0s 91us/step - loss: 46.5359 - val_loss: 50.5511
Epoch 149/1000
274/274 [==============================] - 0s 84us/step - loss: 44.8812 - val_loss: 48.7982
Epoch 150/1000
274/274 [==============================] - 0s 87us/step - loss: 43.3284 - val_loss: 47.1012
Epoch 151/1000
274/274 [==============================] - 0s 87us/step - loss: 41.8442 - val_loss: 45.4683
Epoch 152/1000
274/274 [=============

274/274 [==============================] - 0s 80us/step - loss: 17.2805 - val_loss: 18.1820
Epoch 220/1000
274/274 [==============================] - 0s 80us/step - loss: 17.2390 - val_loss: 18.1359
Epoch 221/1000
274/274 [==============================] - 0s 80us/step - loss: 17.1983 - val_loss: 18.0957
Epoch 222/1000
274/274 [==============================] - 0s 84us/step - loss: 17.1651 - val_loss: 18.0498
Epoch 223/1000
274/274 [==============================] - 0s 73us/step - loss: 17.1264 - val_loss: 18.0053
Epoch 224/1000
274/274 [==============================] - 0s 84us/step - loss: 17.0877 - val_loss: 17.9642
Epoch 225/1000
274/274 [==============================] - 0s 84us/step - loss: 17.0565 - val_loss: 17.9235
Epoch 226/1000
274/274 [==============================] - 0s 76us/step - loss: 17.0234 - val_loss: 17.8799
Epoch 227/1000
274/274 [==============================] - 0s 84us/step - loss: 16.9852 - val_loss: 17.8426
Epoch 228/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 15.3947 - val_loss: 16.2358
Epoch 296/1000
274/274 [==============================] - 0s 80us/step - loss: 15.3760 - val_loss: 16.2212
Epoch 297/1000
274/274 [==============================] - 0s 73us/step - loss: 15.3578 - val_loss: 16.2072
Epoch 298/1000
274/274 [==============================] - 0s 84us/step - loss: 15.3398 - val_loss: 16.1898
Epoch 299/1000
274/274 [==============================] - 0s 80us/step - loss: 15.3229 - val_loss: 16.1732
Epoch 300/1000
274/274 [==============================] - 0s 80us/step - loss: 15.3042 - val_loss: 16.1590
Epoch 301/1000
274/274 [==============================] - 0s 76us/step - loss: 15.2861 - val_loss: 16.1438
Epoch 302/1000
274/274 [==============================] - 0s 76us/step - loss: 15.2697 - val_loss: 16.1297
Epoch 303/1000
274/274 [==============================] - 0s 80us/step - loss: 15.2519 - val_loss: 16.1139
Epoch 304/1000
274/274 [============================

...

274/274 [==============================] - 0s 91us/step - loss: 11.3295 - val_loss: 13.1256
Epoch 752/1000
274/274 [==============================] - 0s 84us/step - loss: 11.3242 - val_loss: 13.1202
Epoch 753/1000
274/274 [==============================] - 0s 87us/step - loss: 11.3196 - val_loss: 13.1154
Epoch 754/1000
274/274 [==============================] - 0s 87us/step - loss: 11.3149 - val_loss: 13.1125
Epoch 755/1000
274/274 [==============================] - 0s 93us/step - loss: 11.3093 - val_loss: 13.1071
Epoch 756/1000
274/274 [==============================] - 0s 88us/step - loss: 11.3044 - val_loss: 13.1011
Epoch 757/1000
274/274 [==============================] - 0s 92us/step - loss: 11.2993 - val_loss: 13.0981
Epoch 758/1000
274/274 [==============================] - 0s 86us/step - loss: 11.2956 - val_loss: 13.0909
Epoch 759/1000
274/274 [==============================] - 0s 87us/step - loss: 11.2899 - val_loss: 13.0865
Epoch 760/1000
274/274 [============================

274/274 [==============================] - 0s 76us/step - loss: 10.9708 - val_loss: 12.8164
Epoch 828/1000
274/274 [==============================] - 0s 80us/step - loss: 10.9669 - val_loss: 12.8130
Epoch 829/1000
274/274 [==============================] - 0s 80us/step - loss: 10.9627 - val_loss: 12.8085
Epoch 830/1000
274/274 [==============================] - 0s 84us/step - loss: 10.9575 - val_loss: 12.8057
Epoch 831/1000
274/274 [==============================] - 0s 76us/step - loss: 10.9531 - val_loss: 12.8037
Epoch 832/1000
274/274 [==============================] - 0s 87us/step - loss: 10.9492 - val_loss: 12.8011
Epoch 833/1000
274/274 [==============================] - 0s 76us/step - loss: 10.9443 - val_loss: 12.7970
Epoch 834/1000
274/274 [==============================] - 0s 88us/step - loss: 10.9412 - val_loss: 12.7923
Epoch 835/1000
274/274 [==============================] - 0s 76us/step - loss: 10.9375 - val_loss: 12.7870
Epoch 836/1000
274/274 [============================

274/274 [==============================] - 0s 76us/step - loss: 10.6573 - val_loss: 12.5455
Epoch 904/1000
274/274 [==============================] - 0s 76us/step - loss: 10.6532 - val_loss: 12.5415
Epoch 905/1000
274/274 [==============================] - 0s 80us/step - loss: 10.6499 - val_loss: 12.5340
Epoch 906/1000
274/274 [==============================] - 0s 80us/step - loss: 10.6459 - val_loss: 12.5313
Epoch 907/1000
274/274 [==============================] - 0s 84us/step - loss: 10.6422 - val_loss: 12.5276
Epoch 908/1000
274/274 [==============================] - 0s 84us/step - loss: 10.6385 - val_loss: 12.5257
Epoch 909/1000
274/274 [==============================] - 0s 77us/step - loss: 10.6352 - val_loss: 12.5218
Epoch 910/1000
274/274 [==============================] - 0s 80us/step - loss: 10.6321 - val_loss: 12.5192
Epoch 911/1000
274/274 [==============================] - 0s 76us/step - loss: 10.6280 - val_loss: 12.5143
Epoch 912/1000
274/274 [============================

274/274 [==============================] - 0s 76us/step - loss: 10.3872 - val_loss: 12.3313
Epoch 980/1000
274/274 [==============================] - 0s 73us/step - loss: 10.3836 - val_loss: 12.3295
Epoch 981/1000
274/274 [==============================] - 0s 80us/step - loss: 10.3805 - val_loss: 12.3258
Epoch 982/1000
274/274 [==============================] - 0s 80us/step - loss: 10.3773 - val_loss: 12.3221
Epoch 983/1000
274/274 [==============================] - 0s 80us/step - loss: 10.3740 - val_loss: 12.3211
Epoch 984/1000
274/274 [==============================] - 0s 76us/step - loss: 10.3697 - val_loss: 12.3170
Epoch 985/1000
274/274 [==============================] - 0s 84us/step - loss: 10.3675 - val_loss: 12.3109
Epoch 986/1000
274/274 [==============================] - 0s 80us/step - loss: 10.3630 - val_loss: 12.3092
Epoch 987/1000
274/274 [==============================] - 0s 73us/step - loss: 10.3605 - val_loss: 12.3053
Epoch 988/1000
274/274 [============================

[array([[-0.05231114,  0.12651244,  0.1263762 ,  0.19615713, -0.02172406],
        [-0.19536307,  0.69421196,  0.2664603 , -0.282068  , -0.4209428 ],
        [-0.22388889,  0.19541124, -0.00980441, -0.27860108,  0.0907437 ],
        [-0.54777503,  0.50613326, -0.14097813, -0.09462523,  0.44924748],
        [ 0.24199961, -0.07165147, -0.03769145, -0.17112769,  0.08668526],
        [ 0.3684564 , -0.3578332 ,  0.42051533, -0.10433065,  0.06810463],
        [-0.17008634,  0.41319373,  0.41348305, -0.10338556, -0.42596078]],
       dtype=float32),
 array([-0.28607512,  0.07373808,  0.7021193 ,  1.445035  , -0.82618344],
       dtype=float32),
 array([[ 0.11193867,  0.25450215,  0.15796387,  0.4847849 ,  0.01999067,
          0.5958489 ,  0.50807947, -0.87742466,  0.65304977, -0.07690794],
        [ 0.95312995,  0.44104365,  0.7380521 , -0.70703155, -0.26138154,
          0.33032554, -0.35604367, -0.29163462, -0.42266688, -0.17106968],
        [-0.64252096, -0.1818912 , -0.7343772 ,  0.31931

In [30]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sgd_2nd.h5')

In [31]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, RMSprop, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 585us/step - loss:

274/274 [==============================] - 0s 89us/step - loss: 8.1536 - val_loss: 11.1927
Epoch 69/1000
274/274 [==============================] - 0s 95us/step - loss: 8.3945 - val_loss: 11.0135
Epoch 70/1000
274/274 [==============================] - 0s 87us/step - loss: 8.1429 - val_loss: 11.8989
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 7.4501 - val_loss: 11.5466
Epoch 72/1000
274/274 [==============================] - 0s 91us/step - loss: 9.0887 - val_loss: 12.2431
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8116 - val_loss: 12.6843
Epoch 74/1000
274/274 [==============================] - 0s 95us/step - loss: 8.2651 - val_loss: 10.6980
Epoch 75/1000
274/274 [==============================] - 0s 88us/step - loss: 7.8563 - val_loss: 12.1037
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 8.7567 - val_loss: 12.3213
Epoch 77/1000
274/274 [==============================] - 0s 95us/step

274/274 [==============================] - 0s 91us/step - loss: 8.3127 - val_loss: 10.3626
Epoch 146/1000
274/274 [==============================] - 0s 87us/step - loss: 7.6044 - val_loss: 9.6959
Epoch 147/1000
274/274 [==============================] - 0s 92us/step - loss: 7.6634 - val_loss: 10.3698
Epoch 148/1000
274/274 [==============================] - 0s 87us/step - loss: 7.8976 - val_loss: 13.8011
Epoch 149/1000
274/274 [==============================] - 0s 87us/step - loss: 7.7619 - val_loss: 10.9651
Epoch 150/1000
274/274 [==============================] - 0s 84us/step - loss: 7.6176 - val_loss: 10.3348
Epoch 151/1000
274/274 [==============================] - 0s 87us/step - loss: 7.9526 - val_loss: 9.9125
Epoch 152/1000
274/274 [==============================] - 0s 87us/step - loss: 7.7723 - val_loss: 10.3410
Epoch 153/1000
274/274 [==============================] - 0s 87us/step - loss: 7.7698 - val_loss: 10.0305
Epoch 154/1000
274/274 [==============================] - 0s 80

274/274 [==============================] - 0s 124us/step - loss: 7.5207 - val_loss: 9.6800
Epoch 223/1000
274/274 [==============================] - 0s 95us/step - loss: 7.5797 - val_loss: 9.6567
Epoch 224/1000
274/274 [==============================] - 0s 91us/step - loss: 7.1556 - val_loss: 10.8420
Epoch 225/1000
274/274 [==============================] - 0s 84us/step - loss: 7.4360 - val_loss: 11.8280
Epoch 226/1000
274/274 [==============================] - 0s 87us/step - loss: 7.6266 - val_loss: 9.7713
Epoch 227/1000
274/274 [==============================] - 0s 76us/step - loss: 7.3211 - val_loss: 10.6839
Epoch 228/1000
274/274 [==============================] - 0s 84us/step - loss: 7.4949 - val_loss: 9.0796
Epoch 229/1000
274/274 [==============================] - 0s 87us/step - loss: 6.9389 - val_loss: 11.1518
Epoch 230/1000
274/274 [==============================] - 0s 84us/step - loss: 8.0828 - val_loss: 9.2905
Epoch 231/1000
274/274 [==============================] - 0s 91us

Epoch 300/1000
274/274 [==============================] - 0s 98us/step - loss: 7.3837 - val_loss: 11.1868
Epoch 301/1000
274/274 [==============================] - 0s 87us/step - loss: 6.9123 - val_loss: 8.9870
Epoch 302/1000
274/274 [==============================] - 0s 91us/step - loss: 7.3049 - val_loss: 11.6140
Epoch 303/1000
274/274 [==============================] - 0s 91us/step - loss: 7.0606 - val_loss: 10.8742
Epoch 304/1000
274/274 [==============================] - 0s 91us/step - loss: 7.2317 - val_loss: 10.4831
Epoch 305/1000
274/274 [==============================] - 0s 95us/step - loss: 7.2852 - val_loss: 10.2954
Epoch 306/1000
274/274 [==============================] - 0s 87us/step - loss: 7.2045 - val_loss: 9.5540
Epoch 307/1000
274/274 [==============================] - 0s 63us/step - loss: 7.3731 - val_loss: 9.5298
Epoch 308/1000
274/274 [==============================] - 0s 132us/step - loss: 6.9836 - val_loss: 8.7036
Epoch 309/1000
274/274 [=========================

...

Epoch 767/1000
274/274 [==============================] - 0s 91us/step - loss: 6.0168 - val_loss: 9.1157
Epoch 768/1000
274/274 [==============================] - 0s 87us/step - loss: 5.8510 - val_loss: 9.0286
Epoch 769/1000
274/274 [==============================] - 0s 91us/step - loss: 6.0009 - val_loss: 9.7244
Epoch 770/1000
274/274 [==============================] - 0s 87us/step - loss: 5.9414 - val_loss: 8.4992
Epoch 771/1000
274/274 [==============================] - 0s 91us/step - loss: 6.2210 - val_loss: 8.7234
Epoch 772/1000
274/274 [==============================] - 0s 84us/step - loss: 5.9653 - val_loss: 8.5936
Epoch 773/1000
274/274 [==============================] - 0s 87us/step - loss: 6.0024 - val_loss: 9.1151
Epoch 774/1000
274/274 [==============================] - 0s 80us/step - loss: 6.2824 - val_loss: 8.8959
Epoch 775/1000
274/274 [==============================] - 0s 91us/step - loss: 5.7683 - val_loss: 8.6190
Epoch 776/1000
274/274 [==============================]

Epoch 845/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7782 - val_loss: 8.9968
Epoch 846/1000
274/274 [==============================] - 0s 106us/step - loss: 5.9879 - val_loss: 9.1026
Epoch 847/1000
274/274 [==============================] - 0s 92us/step - loss: 6.1445 - val_loss: 8.7132
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8693 - val_loss: 9.1141
Epoch 849/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0380 - val_loss: 9.5194
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9198 - val_loss: 8.9863
Epoch 851/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8172 - val_loss: 8.8852
Epoch 852/1000
274/274 [==============================] - 0s 94us/step - loss: 5.7758 - val_loss: 10.6400
Epoch 853/1000
274/274 [==============================] - 0s 81us/step - loss: 6.3044 - val_loss: 8.7398
Epoch 854/1000
274/274 [=============================

Epoch 923/1000
274/274 [==============================] - 0s 85us/step - loss: 6.0737 - val_loss: 8.3528
Epoch 924/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5648 - val_loss: 10.0440
Epoch 925/1000
274/274 [==============================] - 0s 87us/step - loss: 5.6979 - val_loss: 8.5921
Epoch 926/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8982 - val_loss: 8.6531
Epoch 927/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0072 - val_loss: 9.5390
Epoch 928/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0086 - val_loss: 9.1932
Epoch 929/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6976 - val_loss: 10.7205
Epoch 930/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2003 - val_loss: 9.6708
Epoch 931/1000
274/274 [==============================] - 0s 90us/step - loss: 6.0743 - val_loss: 9.2470
Epoch 932/1000
274/274 [============================

8.051487356929456


[array([[ 2.6690612e+00, -1.3844771e+00,  4.2985344e+00, -6.1107439e-01,
          1.9184650e+00],
        [ 4.4172913e-01,  2.2681342e-01, -2.5795622e+00, -4.1228724e-01,
          8.9761323e-01],
        [-6.0513115e-01,  1.8272872e-01, -4.1284120e-01,  9.2792642e-01,
         -9.2501849e-01],
        [-4.8817584e-01,  7.3982441e-01,  3.3558922e+00,  4.8050916e-01,
         -4.2854241e-01],
        [-2.4726285e-01,  7.5060654e-01, -1.6467766e-01, -6.7938668e-01,
          1.0342709e-01],
        [ 1.3066553e+00, -3.7906799e-03, -1.1319046e+00,  5.8794641e-01,
         -2.0383933e+00],
        [-2.1709418e-01, -1.6860880e+00, -3.3433771e-01,  7.6606721e-02,
         -7.8307325e-01]], dtype=float32),
 array([ 1.6809769 , -1.7781898 ,  0.82567185,  0.5865976 , -2.1910431 ],
       dtype=float32),
 array([[ 0.5663346 , -0.75370795, -0.90363955, -0.8124399 ,  0.33290014,
         -0.16958655, -0.7285154 ,  0.8905484 ,  0.08870707,  0.82570344],
        [ 0.04166774,  0.60063064,  0.209687

In [32]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_rmsprop_2nd.h5')

In [33]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 112us/step - loss: 0.0068 - val_loss: 0.0048
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0071 - val_loss: 0.0064
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0042
Epoch 72/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0058 - val_loss: 0.0062
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0062 - val_loss: 0.0043
Epoch 74/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0062 - val_loss: 0.0047
Epoch 75/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0059 - val_loss: 0.0049
Epoch 76/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0057 - val_loss: 0.0048
Epoch 77/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0054 - val_loss: 0.0048
Epoch 78/1000
143/143 [==============================] - 0s 113us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0048
Epoch 147/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0041 - val_loss: 0.0047
Epoch 148/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0045 - val_loss: 0.0043
Epoch 149/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0052 - val_loss: 0.0098
Epoch 150/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0055 - val_loss: 0.0042
Epoch 151/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0046 - val_loss: 0.0048
Epoch 152/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0040 - val_loss: 0.0044
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0034
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0039 - val_loss: 0.0057
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0030 - val_loss: 0.0033
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0043
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0041
Epoch 226/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0034
Epoch 227/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0033 - val_loss: 0.0065
Epoch 228/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0034 - val_loss: 0.0036
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0035
Epoch 230/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0030 - val_loss: 0.0058
Epoch 231/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0036 - val_loss: 0.0033
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0046
Epoch 301/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0027 - val_loss: 0.0055
Epoch 302/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0033 - val_loss: 0.0044
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0038 - val_loss: 0.0075
Epoch 304/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0043 - val_loss: 0.0043
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0043
Epoch 306/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0035 - val_loss: 0.0060
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 308/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0039 - val_loss: 0.0055
Epoch 309/1000
143/143 [==============================] - 0s 1

...

143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0049
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0042
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0062
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0049
Epoch 766/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0020 - val_loss: 0.0044
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0044
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0056
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0020 - val_loss: 0.0048
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0043
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0051
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0050
Epoch 841/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0018 - val_loss: 0.0047
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0045
Epoch 843/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0049
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0062
Epoch 845/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0023 - val_loss: 0.0046
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0046
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0049
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0044
Epoch 918/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 919/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0016 - val_loss: 0.0048
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0047
Epoch 921/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0048
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0047
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0015 - val_loss: 0.0049
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0048
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0048
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0043
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0056
Epoch 996/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0020 - val_loss: 0.0042
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 998/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0020 - val_loss: 0.0052
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 1000/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0016 - val_loss: 0.0043
0.01863112300634384


[array([[ 1.0959928 , -0.17459916, -0.3729455 ,  0.62501806,  0.31459203],
        [-0.5917074 ,  0.49819738, -0.18975246,  0.3700388 , -0.3825755 ],
        [-1.2930732 , -0.13203529,  0.6449758 , -0.09866732,  0.0682577 ],
        [-0.7899486 ,  0.47234327, -0.41956103,  0.7610962 , -0.23974106],
        [-0.6984903 , -0.26560292,  0.16276215,  0.7598041 , -0.06150069],
        [ 0.9282308 , -0.5278945 , -0.17230275,  0.37509197, -0.5026838 ],
        [-0.3230234 , -0.21592306, -0.26587492,  0.54431283, -0.52996653],
        [-0.06379219,  0.29524288, -0.47673675,  0.30279356,  0.10567314],
        [ 0.36372826, -0.75964344,  0.6751306 ,  0.45946944,  0.11237138],
        [ 0.18515678,  0.4450676 , -0.18133095,  0.11029539,  0.16288354],
        [-0.60335475,  0.18922828,  1.0872743 , -0.9395155 ,  0.04866156],
        [ 0.1514007 ,  0.4431238 ,  0.1500728 , -0.6002698 ,  0.10987886],
        [ 0.21136391,  0.27824438,  0.04943134, -1.0458444 ,  0.3623466 ],
        [ 2.2048602 , -0.

In [34]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_adam_2nd.h5')

In [35]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, sgd, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 105us/step - loss: 0.6006 - val_loss: 0.5147
Epoch 70/1000
143/143 [==============================] - 0s 91us/step - loss: 0.5986 - val_loss: 0.5129
Epoch 71/1000
143/143 [==============================] - 0s 98us/step - loss: 0.5966 - val_loss: 0.5112
Epoch 72/1000
143/143 [==============================] - 0s 98us/step - loss: 0.5946 - val_loss: 0.5094
Epoch 73/1000
143/143 [==============================] - 0s 91us/step - loss: 0.5926 - val_loss: 0.5076
Epoch 74/1000
143/143 [==============================] - 0s 91us/step - loss: 0.5906 - val_loss: 0.5059
Epoch 75/1000
143/143 [==============================] - 0s 105us/step - loss: 0.5886 - val_loss: 0.5041
Epoch 76/1000
143/143 [==============================] - 0s 91us/step - loss: 0.5867 - val_loss: 0.5024
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.5847 - val_loss: 0.5007
Epoch 78/1000
143/143 [==============================] - 0s 84us/step - loss

143/143 [==============================] - 0s 91us/step - loss: 0.4674 - val_loss: 0.3973
Epoch 148/1000
143/143 [==============================] - 0s 98us/step - loss: 0.4660 - val_loss: 0.3960
Epoch 149/1000
143/143 [==============================] - 0s 91us/step - loss: 0.4646 - val_loss: 0.3948
Epoch 150/1000
143/143 [==============================] - 0s 98us/step - loss: 0.4632 - val_loss: 0.3936
Epoch 151/1000
143/143 [==============================] - 0s 91us/step - loss: 0.4618 - val_loss: 0.3923
Epoch 152/1000
143/143 [==============================] - 0s 98us/step - loss: 0.4604 - val_loss: 0.3911
Epoch 153/1000
143/143 [==============================] - 0s 91us/step - loss: 0.4590 - val_loss: 0.3899
Epoch 154/1000
143/143 [==============================] - 0s 91us/step - loss: 0.4576 - val_loss: 0.3887
Epoch 155/1000
143/143 [==============================] - 0s 91us/step - loss: 0.4563 - val_loss: 0.3875
Epoch 156/1000
143/143 [==============================] - 0s 91us/step

143/143 [==============================] - 0s 84us/step - loss: 0.3739 - val_loss: 0.3158
Epoch 226/1000
143/143 [==============================] - 0s 91us/step - loss: 0.3729 - val_loss: 0.3149
Epoch 227/1000
143/143 [==============================] - 0s 84us/step - loss: 0.3719 - val_loss: 0.3141
Epoch 228/1000
143/143 [==============================] - 0s 91us/step - loss: 0.3709 - val_loss: 0.3132
Epoch 229/1000
143/143 [==============================] - 0s 84us/step - loss: 0.3699 - val_loss: 0.3124
Epoch 230/1000
143/143 [==============================] - 0s 98us/step - loss: 0.3689 - val_loss: 0.3115
Epoch 231/1000
143/143 [==============================] - 0s 84us/step - loss: 0.3680 - val_loss: 0.3106
Epoch 232/1000
143/143 [==============================] - 0s 91us/step - loss: 0.3669 - val_loss: 0.3098
Epoch 233/1000
143/143 [==============================] - 0s 90us/step - loss: 0.3660 - val_loss: 0.3089
Epoch 234/1000
143/143 [==============================] - 0s 83us/step

143/143 [==============================] - 0s 98us/step - loss: 0.3075 - val_loss: 0.2587
Epoch 304/1000
143/143 [==============================] - 0s 99us/step - loss: 0.3068 - val_loss: 0.2581
Epoch 305/1000
143/143 [==============================] - 0s 91us/step - loss: 0.3061 - val_loss: 0.2575
Epoch 306/1000
143/143 [==============================] - 0s 98us/step - loss: 0.3053 - val_loss: 0.2569
Epoch 307/1000
143/143 [==============================] - 0s 98us/step - loss: 0.3046 - val_loss: 0.2563
Epoch 308/1000
143/143 [==============================] - 0s 98us/step - loss: 0.3039 - val_loss: 0.2556
Epoch 309/1000
143/143 [==============================] - 0s 91us/step - loss: 0.3032 - val_loss: 0.2550
Epoch 310/1000
143/143 [==============================] - 0s 84us/step - loss: 0.3025 - val_loss: 0.2544
Epoch 311/1000
143/143 [==============================] - 0s 98us/step - loss: 0.3018 - val_loss: 0.2538
Epoch 312/1000
143/143 [==============================] - 0s 91us/step

...

Epoch 692/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1660 - val_loss: 0.1417
Epoch 693/1000
143/143 [==============================] - 0s 98us/step - loss: 0.1658 - val_loss: 0.1415
Epoch 694/1000
143/143 [==============================] - 0s 84us/step - loss: 0.1657 - val_loss: 0.1414
Epoch 695/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1655 - val_loss: 0.1413
Epoch 696/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1653 - val_loss: 0.1412
Epoch 697/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1652 - val_loss: 0.1410
Epoch 698/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1650 - val_loss: 0.1409
Epoch 699/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1648 - val_loss: 0.1408
Epoch 700/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1647 - val_loss: 0.1407
Epoch 701/1000
143/143 [==============================]

Epoch 770/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1551 - val_loss: 0.1333
Epoch 771/1000
143/143 [==============================] - 0s 104us/step - loss: 0.1550 - val_loss: 0.1332
Epoch 772/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1549 - val_loss: 0.1331
Epoch 773/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1548 - val_loss: 0.1330
Epoch 774/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1546 - val_loss: 0.1329
Epoch 775/1000
143/143 [==============================] - 0s 98us/step - loss: 0.1545 - val_loss: 0.1328
Epoch 776/1000
143/143 [==============================] - 0s 98us/step - loss: 0.1544 - val_loss: 0.1327
Epoch 777/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1543 - val_loss: 0.1326
Epoch 778/1000
143/143 [==============================] - 0s 98us/step - loss: 0.1542 - val_loss: 0.1325
Epoch 779/1000
143/143 [==============================

Epoch 848/1000
143/143 [==============================] - 0s 84us/step - loss: 0.1467 - val_loss: 0.1269
Epoch 849/1000
143/143 [==============================] - 0s 84us/step - loss: 0.1466 - val_loss: 0.1268
Epoch 850/1000
143/143 [==============================] - 0s 98us/step - loss: 0.1465 - val_loss: 0.1268
Epoch 851/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1464 - val_loss: 0.1267
Epoch 852/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1463 - val_loss: 0.1266
Epoch 853/1000
143/143 [==============================] - 0s 84us/step - loss: 0.1462 - val_loss: 0.1265
Epoch 854/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1462 - val_loss: 0.1265
Epoch 855/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1461 - val_loss: 0.1264
Epoch 856/1000
143/143 [==============================] - 0s 84us/step - loss: 0.1460 - val_loss: 0.1263
Epoch 857/1000
143/143 [==============================]

143/143 [==============================] - 0s 98us/step - loss: 0.1402 - val_loss: 0.1220
Epoch 927/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1401 - val_loss: 0.1220
Epoch 928/1000
143/143 [==============================] - 0s 98us/step - loss: 0.1401 - val_loss: 0.1219
Epoch 929/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1400 - val_loss: 0.1219
Epoch 930/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1399 - val_loss: 0.1218
Epoch 931/1000
143/143 [==============================] - 0s 84us/step - loss: 0.1399 - val_loss: 0.1218
Epoch 932/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1398 - val_loss: 0.1217
Epoch 933/1000
143/143 [==============================] - 0s 91us/step - loss: 0.1397 - val_loss: 0.1217
Epoch 934/1000
143/143 [==============================] - 0s 90us/step - loss: 0.1396 - val_loss: 0.1216
Epoch 935/1000
143/143 [==============================] - 0s 84us/step

[array([[ 0.44749817,  0.43058276, -0.0732604 , -0.2730294 , -0.2135475 ],
        [ 0.273385  , -0.43561307,  0.11358029,  0.1775234 ,  0.22403076],
        [-0.13714333, -0.07853865, -0.23490739, -0.26359332, -0.31931153],
        [-0.15491037, -0.20108978,  0.4479912 , -0.25765964,  0.3646802 ],
        [ 0.17188923,  0.44070795,  0.32161576, -0.0662131 ,  0.04318114],
        [ 0.39512783,  0.34095925, -0.15394492,  0.3282869 , -0.39534953],
        [-0.13171232,  0.12497909, -0.09154812,  0.185897  , -0.43011597],
        [-0.430862  , -0.05901862, -0.23249988, -0.05399773,  0.09468611],
        [ 0.22293434, -0.40188164,  0.19425711,  0.04249582, -0.00817109],
        [-0.1899536 ,  0.22039177,  0.40363556,  0.413685  , -0.02618048],
        [ 0.42016235, -0.389917  ,  0.05256459, -0.3238426 ,  0.22722284],
        [ 0.27789575,  0.17032982,  0.47112378, -0.16949576,  0.31125602],
        [ 0.09705093, -0.25332695,  0.27044493, -0.26531658, -0.32624438],
        [ 0.25816163, -0.

In [36]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sgd_2nd.h5')

In [37]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, RMSprop, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 104us/step - loss: 0.0108 - val_loss: 0.0066
Epoch 70/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0101 - val_loss: 0.0073
Epoch 71/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0132 - val_loss: 0.0112
Epoch 72/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0110 - val_loss: 0.0082
Epoch 73/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0100 - val_loss: 0.0116
Epoch 74/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0091 - val_loss: 0.0067
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0106 - val_loss: 0.0061
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0075 - val_loss: 0.0089
Epoch 77/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0127 - val_loss: 0.0107
Epoch 78/1000
143/143 [==============================] - 0s 112us/step

Epoch 147/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0083 - val_loss: 0.0079
Epoch 148/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0080 - val_loss: 0.0061
Epoch 149/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0066 - val_loss: 0.0084
Epoch 150/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0133 - val_loss: 0.0050
Epoch 151/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0065 - val_loss: 0.0070
Epoch 152/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0080 - val_loss: 0.0078
Epoch 153/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0083 - val_loss: 0.0058
Epoch 154/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0094 - val_loss: 0.0056
Epoch 155/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0097 - val_loss: 0.0102
Epoch 156/1000
143/143 [===========================

143/143 [==============================] - 0s 98us/step - loss: 0.0071 - val_loss: 0.0049
Epoch 225/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0068 - val_loss: 0.0061
Epoch 226/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0097 - val_loss: 0.0058
Epoch 227/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0057 - val_loss: 0.0050
Epoch 228/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0070 - val_loss: 0.0064
Epoch 229/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0059 - val_loss: 0.0059
Epoch 230/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0066 - val_loss: 0.0083
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0065 - val_loss: 0.0068
Epoch 232/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0080 - val_loss: 0.0045
Epoch 233/1000
143/143 [==============================] - 0s 91us/ste

Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0050 - val_loss: 0.0080
Epoch 303/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0056 - val_loss: 0.0112
Epoch 304/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0072 - val_loss: 0.0068
Epoch 305/1000
143/143 [==============================] - ETA: 0s - loss: 0.006 - 0s 104us/step - loss: 0.0061 - val_loss: 0.0050
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0062 - val_loss: 0.0057
Epoch 308/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0056 - val_loss: 0.0046
Epoch 309/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0043 - val_loss: 0.0064
Epoch 310/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0078 - val_loss: 0.0048
Epoch 311/1000
143/143 

...

Epoch 765/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0044
Epoch 766/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0030 - val_loss: 0.0060
Epoch 767/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0030 - val_loss: 0.0034
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 769/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0038 - val_loss: 0.0040
Epoch 770/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0023 - val_loss: 0.0043
Epoch 771/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0029 - val_loss: 0.0049
Epoch 772/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0028 - val_loss: 0.0046
Epoch 773/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0033 - val_loss: 0.0037
Epoch 774/1000
143/143 [=============================

143/143 [==============================] - 0s 98us/step - loss: 0.0037 - val_loss: 0.0036
Epoch 843/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0024 - val_loss: 0.0053
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0032 - val_loss: 0.0032
Epoch 845/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0025 - val_loss: 0.0045
Epoch 846/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0025 - val_loss: 0.0045
Epoch 847/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0034 - val_loss: 0.0035
Epoch 848/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0021 - val_loss: 0.0039
Epoch 849/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0038 - val_loss: 0.0040
Epoch 850/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0036
Epoch 851/1000
143/143 [==============================] - 0s 105us/

143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0032
Epoch 920/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0021 - val_loss: 0.0032
Epoch 921/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0035 - val_loss: 0.0042
Epoch 922/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 923/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0021 - val_loss: 0.0042
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0027 - val_loss: 0.0031
Epoch 925/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0023 - val_loss: 0.0063
Epoch 926/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0039 - val_loss: 0.0030
Epoch 927/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0021 - val_loss: 0.0032
Epoch 928/1000
143/143 [==============================] - 0s 105

143/143 [==============================] - 0s 91us/step - loss: 0.0020 - val_loss: 0.0029
Epoch 997/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0023 - val_loss: 0.0035
Epoch 998/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0046
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0032
Epoch 1000/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0020 - val_loss: 0.0031
0.005499137099832296


[array([[-8.44852448e-01, -2.16377243e-01, -1.11470468e-01,
          1.09479591e-01, -7.87591636e-01],
        [-7.06230164e-01,  5.11918426e-01,  9.60715294e-01,
          5.50078094e-01,  1.02761352e+00],
        [ 1.04486799e+00,  2.19814852e-02,  5.33250690e-01,
         -3.00145239e-01, -4.94944394e-01],
        [ 1.86738908e+00,  6.04151972e-02, -6.11161351e-01,
         -2.71324009e-01, -2.63228249e-02],
        [ 2.30691861e-03,  4.33336169e-01, -3.64422798e-01,
          4.72512633e-01, -6.25574470e-01],
        [-1.22539437e+00,  3.97608638e-01,  1.43978879e-01,
          6.79039657e-02,  2.70576477e-01],
        [-4.96059805e-02,  1.62686482e-01,  7.62848675e-01,
         -1.41359538e-01,  1.72733426e+00],
        [ 7.34506994e-02, -2.67147481e-01,  2.16184035e-01,
         -5.52709438e-02,  1.65337563e-01],
        [ 5.28005123e-01,  1.11413610e+00, -1.62987697e+00,
          1.13631152e-01,  5.28747737e-01],
        [ 1.32993206e-01, -2.01109052e+00,  6.06104076e-01,
    

In [38]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_rmsprop_2nd.h5')

In [39]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 7ms/step - loss: 36.279

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0146 - val_loss: 0.0103
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0124 - val_loss: 0.0104
Epoch 73/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0105 - val_loss: 0.0107
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0094 - val_loss: 0.0112
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0089 - val_loss: 0.0117
Epoch 76/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0089 - val_loss: 0.0122
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0092 - val_loss: 0.0124
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0095 - val_loss: 0.0123
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0097 - val_loss: 0.0119
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0042 - val_loss: 0.0043
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0042 - val_loss: 0.0043
Epoch 153/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0042 - val_loss: 0.0043
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0041 - val_loss: 0.0043
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0041 - val_loss: 0.0043
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0042
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0040 - val_loss: 0.0041
Epoch 160/1000
32/32 [==============================] - 0s 155us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0027
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 0.0027
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0027 - val_loss: 0.0027
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0026 - val_loss: 0.0027
Epoch 234/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0026 - val_loss: 0.0026
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0026 - val_loss: 0.0026
Epoch 236/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0026 - val_loss: 0.0026
Epoch 237/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0026 - val_loss: 0.0026
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0026
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0019 - val_loss: 0.0018
Epoch 310/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0019 - val_loss: 0.0018
Epoch 311/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 314/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 315/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 317/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 318/1000
32/32 [==============================] - 0s 125us

...

Epoch 763/1000
32/32 [==============================] - 0s 124us/step - loss: 5.1829e-04 - val_loss: 3.8143e-04
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1723e-04 - val_loss: 3.8082e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1616e-04 - val_loss: 3.8022e-04
Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1511e-04 - val_loss: 3.7963e-04
Epoch 767/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1405e-04 - val_loss: 3.7903e-04
Epoch 768/1000
32/32 [==============================] - 0s 124us/step - loss: 5.1300e-04 - val_loss: 3.7845e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1196e-04 - val_loss: 3.7786e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1091e-04 - val_loss: 3.7728e-04
Epoch 771/1000
32/32 [==============================] - 0s 125us/step - loss: 5.0988e-04 - val_loss: 3.7

Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4788e-04 - val_loss: 3.4461e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4703e-04 - val_loss: 3.4420e-04
Epoch 839/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4619e-04 - val_loss: 3.4379e-04
Epoch 840/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4534e-04 - val_loss: 3.4340e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4450e-04 - val_loss: 3.4300e-04
Epoch 842/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4367e-04 - val_loss: 3.4260e-04
Epoch 843/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4283e-04 - val_loss: 3.4220e-04
Epoch 844/1000
32/32 [==============================] - 0s 125us/step - loss: 4.4200e-04 - val_loss: 3.4182e-04
Epoch 845/1000
32/32 [==============================] - 0s 156us/step - loss: 4.4117e-04 - val_loss: 3.4

Epoch 911/1000
32/32 [==============================] - 0s 125us/step - loss: 3.9136e-04 - val_loss: 3.1970e-04
Epoch 912/1000
32/32 [==============================] - 0s 125us/step - loss: 3.9067e-04 - val_loss: 3.1942e-04
Epoch 913/1000
32/32 [==============================] - 0s 125us/step - loss: 3.8999e-04 - val_loss: 3.1916e-04
Epoch 914/1000
32/32 [==============================] - 0s 124us/step - loss: 3.8931e-04 - val_loss: 3.1888e-04
Epoch 915/1000
32/32 [==============================] - 0s 124us/step - loss: 3.8863e-04 - val_loss: 3.1861e-04
Epoch 916/1000
32/32 [==============================] - 0s 155us/step - loss: 3.8795e-04 - val_loss: 3.1834e-04
Epoch 917/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8728e-04 - val_loss: 3.1807e-04
Epoch 918/1000
32/32 [==============================] - 0s 156us/step - loss: 3.8660e-04 - val_loss: 3.1781e-04
Epoch 919/1000
32/32 [==============================] - 0s 125us/step - loss: 3.8593e-04 - val_loss: 3.1

Epoch 985/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4535e-04 - val_loss: 3.0250e-04
Epoch 986/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4479e-04 - val_loss: 3.0230e-04
Epoch 987/1000
32/32 [==============================] - 0s 158us/step - loss: 3.4423e-04 - val_loss: 3.0211e-04
Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4367e-04 - val_loss: 3.0191e-04
Epoch 989/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4312e-04 - val_loss: 3.0171e-04
Epoch 990/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4256e-04 - val_loss: 3.0152e-04
Epoch 991/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4200e-04 - val_loss: 3.0134e-04
Epoch 992/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4145e-04 - val_loss: 3.0115e-04
Epoch 993/1000
32/32 [==============================] - 0s 125us/step - loss: 3.4090e-04 - val_loss: 3.0

[array([[-0.08077129, -0.40693408, -0.31906602, -0.4938172 , -0.17201525],
        [-0.9847686 , -0.7185617 ,  1.0327723 , -0.27676013, -0.42201003],
        [ 0.56733036, -0.37597203, -0.5528961 , -1.0427204 ,  0.5739549 ]],
       dtype=float32),
 array([-0.45390186,  0.3492151 ,  0.09672013, -0.39901376, -0.48546174],
       dtype=float32),
 array([[ 0.04652176, -0.45594093, -0.35062853,  0.41902417, -0.31988424,
          0.38528275, -0.44016388,  0.14218436,  0.3370115 ,  0.5326976 ],
        [ 0.38115975, -0.16807574,  0.27257282,  0.39710453, -0.20314553,
          0.25999096,  0.399251  , -0.7723029 , -0.19338493, -0.456151  ],
        [ 0.4294598 ,  0.04155463, -0.47621205,  0.45750394, -0.05033705,
          0.26229903,  0.25685695, -0.16242959,  0.15237947, -0.2903632 ],
        [ 0.38639814,  0.07010184, -0.36703777, -0.11167523,  0.22573768,
          0.03253043, -0.53213   ,  0.07817455, -0.5644139 ,  0.34562764],
        [-0.22079974,  0.20360617, -0.37770274,  0.4257036

In [40]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_adam_2nd.h5')

In [41]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, sgd, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 35.929

Epoch 70/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4858 - val_loss: 35.2146
Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4785 - val_loss: 35.2072
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4713 - val_loss: 35.1998
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4641 - val_loss: 35.1925
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 35.4569 - val_loss: 35.1851
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4496 - val_loss: 35.1777
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4424 - val_loss: 35.1703
Epoch 77/1000
32/32 [==============================] - 0s 124us/step - loss: 35.4352 - val_loss: 35.1629
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 35.4280 - val_loss: 35.1555
Epoch 79/1000
32/32 [==============================] - 

Epoch 148/1000
32/32 [==============================] - 0s 156us/step - loss: 34.9261 - val_loss: 34.6424
Epoch 149/1000
32/32 [==============================] - 0s 125us/step - loss: 34.9190 - val_loss: 34.6351
Epoch 150/1000
32/32 [==============================] - 0s 125us/step - loss: 34.9119 - val_loss: 34.6278
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 34.9048 - val_loss: 34.6206
Epoch 152/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8977 - val_loss: 34.6133
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8906 - val_loss: 34.6060
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8835 - val_loss: 34.5988
Epoch 155/1000
32/32 [==============================] - 0s 93us/step - loss: 34.8764 - val_loss: 34.5915
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8693 - val_loss: 34.5842
Epoch 157/1000
32/32 [=========================

Epoch 226/1000
32/32 [==============================] - 0s 156us/step - loss: 34.3755 - val_loss: 34.0792
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 34.3685 - val_loss: 34.0720
Epoch 228/1000
32/32 [==============================] - 0s 124us/step - loss: 34.3615 - val_loss: 34.0649
Epoch 229/1000
32/32 [==============================] - 0s 125us/step - loss: 34.3545 - val_loss: 34.0577
Epoch 230/1000
32/32 [==============================] - 0s 125us/step - loss: 34.3475 - val_loss: 34.0506
Epoch 231/1000
32/32 [==============================] - 0s 155us/step - loss: 34.3405 - val_loss: 34.0434
Epoch 232/1000
32/32 [==============================] - 0s 124us/step - loss: 34.3335 - val_loss: 34.0363
Epoch 233/1000
32/32 [==============================] - 0s 124us/step - loss: 34.3265 - val_loss: 34.0291
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 34.3195 - val_loss: 34.0219
Epoch 235/1000
32/32 [========================

Epoch 304/1000
32/32 [==============================] - 0s 155us/step - loss: 33.8331 - val_loss: 33.5244
Epoch 305/1000
32/32 [==============================] - 0s 125us/step - loss: 33.8262 - val_loss: 33.5173
Epoch 306/1000
32/32 [==============================] - 0s 125us/step - loss: 33.8193 - val_loss: 33.5103
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 33.8124 - val_loss: 33.5032
Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 33.8055 - val_loss: 33.4961
Epoch 309/1000
32/32 [==============================] - 0s 94us/step - loss: 33.7986 - val_loss: 33.4891
Epoch 310/1000
32/32 [==============================] - 0s 155us/step - loss: 33.7917 - val_loss: 33.4820
Epoch 311/1000
32/32 [==============================] - 0s 93us/step - loss: 33.7848 - val_loss: 33.4750
Epoch 312/1000
32/32 [==============================] - 0s 124us/step - loss: 33.7779 - val_loss: 33.4679
Epoch 313/1000
32/32 [==========================

...

Epoch 694/1000
32/32 [==============================] - 0s 125us/step - loss: 31.2238 - val_loss: 30.8528
Epoch 695/1000
32/32 [==============================] - 0s 156us/step - loss: 31.2173 - val_loss: 30.8461
Epoch 696/1000
32/32 [==============================] - 0s 156us/step - loss: 31.2108 - val_loss: 30.8395
Epoch 697/1000
32/32 [==============================] - 0s 156us/step - loss: 31.2043 - val_loss: 30.8328
Epoch 698/1000
32/32 [==============================] - 0s 156us/step - loss: 31.1978 - val_loss: 30.8261
Epoch 699/1000
32/32 [==============================] - 0s 156us/step - loss: 31.1913 - val_loss: 30.8195
Epoch 700/1000
32/32 [==============================] - 0s 125us/step - loss: 31.1848 - val_loss: 30.8128
Epoch 701/1000
32/32 [==============================] - 0s 94us/step - loss: 31.1783 - val_loss: 30.8062
Epoch 702/1000
32/32 [==============================] - 0s 125us/step - loss: 31.1718 - val_loss: 30.7995
Epoch 703/1000
32/32 [=========================

Epoch 772/1000
32/32 [==============================] - 0s 94us/step - loss: 30.7188 - val_loss: 30.3352
Epoch 773/1000
32/32 [==============================] - 0s 125us/step - loss: 30.7123 - val_loss: 30.3286
Epoch 774/1000
32/32 [==============================] - 0s 124us/step - loss: 30.7059 - val_loss: 30.3220
Epoch 775/1000
32/32 [==============================] - 0s 124us/step - loss: 30.6994 - val_loss: 30.3154
Epoch 776/1000
32/32 [==============================] - 0s 156us/step - loss: 30.6930 - val_loss: 30.3087
Epoch 777/1000
32/32 [==============================] - 0s 125us/step - loss: 30.6865 - val_loss: 30.3021
Epoch 778/1000
32/32 [==============================] - 0s 125us/step - loss: 30.6801 - val_loss: 30.2955
Epoch 779/1000
32/32 [==============================] - 0s 125us/step - loss: 30.6737 - val_loss: 30.2889
Epoch 780/1000
32/32 [==============================] - 0s 125us/step - loss: 30.6672 - val_loss: 30.2823
Epoch 781/1000
32/32 [=========================

Epoch 850/1000
32/32 [==============================] - 0s 125us/step - loss: 30.2183 - val_loss: 29.8220
Epoch 851/1000
32/32 [==============================] - 0s 124us/step - loss: 30.2119 - val_loss: 29.8155
Epoch 852/1000
32/32 [==============================] - 0s 125us/step - loss: 30.2055 - val_loss: 29.8089
Epoch 853/1000
32/32 [==============================] - 0s 124us/step - loss: 30.1991 - val_loss: 29.8024
Epoch 854/1000
32/32 [==============================] - 0s 125us/step - loss: 30.1927 - val_loss: 29.7958
Epoch 855/1000
32/32 [==============================] - 0s 156us/step - loss: 30.1863 - val_loss: 29.7893
Epoch 856/1000
32/32 [==============================] - 0s 124us/step - loss: 30.1799 - val_loss: 29.7827
Epoch 857/1000
32/32 [==============================] - 0s 94us/step - loss: 30.1736 - val_loss: 29.7762
Epoch 858/1000
32/32 [==============================] - 0s 94us/step - loss: 30.1672 - val_loss: 29.7696
Epoch 859/1000
32/32 [==========================

Epoch 928/1000
32/32 [==============================] - 0s 124us/step - loss: 29.7220 - val_loss: 29.3130
Epoch 929/1000
32/32 [==============================] - 0s 124us/step - loss: 29.7156 - val_loss: 29.3065
Epoch 930/1000
32/32 [==============================] - 0s 156us/step - loss: 29.7093 - val_loss: 29.3000
Epoch 931/1000
32/32 [==============================] - 0s 124us/step - loss: 29.7029 - val_loss: 29.2935
Epoch 932/1000
32/32 [==============================] - 0s 125us/step - loss: 29.6966 - val_loss: 29.2870
Epoch 933/1000
32/32 [==============================] - 0s 94us/step - loss: 29.6903 - val_loss: 29.2805
Epoch 934/1000
32/32 [==============================] - 0s 156us/step - loss: 29.6839 - val_loss: 29.2740
Epoch 935/1000
32/32 [==============================] - 0s 93us/step - loss: 29.6776 - val_loss: 29.2675
Epoch 936/1000
32/32 [==============================] - 0s 125us/step - loss: 29.6713 - val_loss: 29.2610
Epoch 937/1000
32/32 [==========================

[array([[ 0.53952616, -0.59826696,  0.5644748 , -0.29888788, -0.04832367],
        [ 0.00756492,  0.25885287,  0.61467445, -0.82407767, -0.31862628],
        [ 0.67316675, -0.44051468,  0.3385642 ,  0.76385105, -0.56279504]],
       dtype=float32),
 array([-0.04846346,  0.01743151, -0.02789615, -0.03060336, -0.09740838],
       dtype=float32),
 array([[ 0.387891  ,  0.37094423,  0.4332685 , -0.17098673, -0.45252723,
          0.6216626 ,  0.6156873 ,  0.28232935,  0.25684056, -0.15465258],
        [ 0.1340369 , -0.114769  , -0.1210269 , -0.5154139 ,  0.45345712,
          0.04828865,  0.17190395,  0.15790547,  0.2997108 ,  0.0544431 ],
        [ 0.5853485 , -0.06022786,  0.14791024,  0.0783567 , -0.52164745,
          0.5962962 , -0.42832097,  0.60805064, -0.37891352, -0.4445496 ],
        [-0.0065888 ,  0.04226655,  0.21924162,  0.5073278 , -0.3401079 ,
          0.26190537,  0.20450655, -0.53362095,  0.12633921, -0.38349697],
        [-0.4868068 ,  0.09785797, -0.55011773, -0.4023399

In [42]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sgd_2nd.h5')

In [43]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, RMSprop, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 7ms/step - loss: 39.304

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0564 - val_loss: 0.0441
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0512 - val_loss: 0.0342
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0533 - val_loss: 0.0447
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0507 - val_loss: 0.0403
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0539 - val_loss: 0.0463
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0509 - val_loss: 0.0435
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0539 - val_loss: 0.0472
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0505 - val_loss: 0.0440
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0535 - val_loss: 0.0473
Epoch 80/1000
32/32 [==============================] - 0s 93us/step - los

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0556 - val_loss: 0.0307
Epoch 152/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0629 - val_loss: 0.0252
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0785 - val_loss: 0.0280
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0665 - val_loss: 0.0203
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0526 - val_loss: 0.0203
Epoch 156/1000
32/32 [==============================] - 0s 135us/step - loss: 0.0381 - val_loss: 0.0224
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0346 - val_loss: 0.0207
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0325 - val_loss: 0.0333
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0375 - val_loss: 0.0300
Epoch 160/1000
32/32 [==============================] - 0s 93us/

Epoch 230/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0412 - val_loss: 0.0544
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0474 - val_loss: 0.0420
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0438 - val_loss: 0.0600
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0457 - val_loss: 0.0451
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0398 - val_loss: 0.0593
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0408 - val_loss: 0.0454
Epoch 236/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0363 - val_loss: 0.0571
Epoch 237/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0379 - val_loss: 0.0443
Epoch 238/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0348 - val_loss: 0.0546
Epoch 239/1000
32/32 [==============================] - 0s 125us/s

Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0342 - val_loss: 0.0413
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0327 - val_loss: 0.0514
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0375 - val_loss: 0.0407
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0391 - val_loss: 0.0466
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0489 - val_loss: 0.0310
Epoch 314/1000
32/32 [==============================] - 0s 120us/step - loss: 0.0537 - val_loss: 0.0306
Epoch 315/1000
32/32 [==============================] - 0s 123us/step - loss: 0.0687 - val_loss: 0.0219
Epoch 316/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0637 - val_loss: 0.0215
Epoch 317/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0529 - val_loss: 0.0139
Epoch 318/1000
32/32 [==============================] - 0s 125us

...

Epoch 704/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0233 - val_loss: 0.0421
Epoch 705/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0596 - val_loss: 0.1048
Epoch 706/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1211 - val_loss: 0.0791
Epoch 707/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0949 - val_loss: 0.0638
Epoch 708/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0650 - val_loss: 0.0244
Epoch 709/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0279 - val_loss: 0.0176
Epoch 710/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0168 - val_loss: 0.0082
Epoch 711/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0099 - val_loss: 0.0080
Epoch 712/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0078 - val_loss: 0.0048
Epoch 713/1000
32/32 [==============================] - 0s 125us/

Epoch 783/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0416 - val_loss: 0.0410
Epoch 784/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0359 - val_loss: 0.0497
Epoch 785/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0362 - val_loss: 0.0404
Epoch 786/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0302 - val_loss: 0.0456
Epoch 787/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0302 - val_loss: 0.0352
Epoch 788/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0257 - val_loss: 0.0375
Epoch 789/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0258 - val_loss: 0.0273
Epoch 790/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0228 - val_loss: 0.0302
Epoch 791/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0241 - val_loss: 0.0217
Epoch 792/1000
32/32 [==============================] - 0s 125us

Epoch 862/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0068 - val_loss: 0.0111
Epoch 863/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0087 - val_loss: 0.0211
Epoch 864/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0138 - val_loss: 0.0314
Epoch 865/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0225 - val_loss: 0.0562
Epoch 866/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0380 - val_loss: 0.0602
Epoch 867/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0468 - val_loss: 0.0683
Epoch 868/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0530 - val_loss: 0.0457
Epoch 869/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0409 - val_loss: 0.0432
Epoch 870/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0355 - val_loss: 0.0289
Epoch 871/1000
32/32 [==============================] - 0s 156us/

Epoch 941/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0223 - val_loss: 0.0185
Epoch 942/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0240 - val_loss: 0.0139
Epoch 943/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0241 - val_loss: 0.0174
Epoch 944/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0247 - val_loss: 0.0125
Epoch 945/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0227 - val_loss: 0.0144
Epoch 946/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0205 - val_loss: 0.0100
Epoch 947/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0176 - val_loss: 0.0129
Epoch 948/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0162 - val_loss: 0.0116
Epoch 949/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0165 - val_loss: 0.0223
Epoch 950/1000
32/32 [==============================] - 0s 155us

[array([[-0.71458894, -0.24896567, -0.1649349 ,  0.14206788,  0.34564862],
        [ 0.32734793,  0.03354254, -0.43399936, -0.24522617, -0.0046199 ],
        [-0.37366173,  0.08916539,  0.40448037, -0.34349647, -1.1836096 ]],
       dtype=float32),
 array([ 0.12872593, -0.4126139 , -0.21995746,  0.47616103, -0.2330058 ],
       dtype=float32),
 array([[ 0.08445498, -0.00715817,  0.20756647, -0.20227155, -0.21300034,
          0.8139194 , -0.25034007,  0.19535618,  0.5497829 ,  0.19385365],
        [ 0.33245674,  0.03375164,  0.29085335,  0.15033722,  0.350774  ,
         -0.395077  , -0.01142964, -0.36585945, -0.2643435 , -0.59149694],
        [ 0.14696895,  0.03544979,  0.47984785, -0.29932562, -0.27350557,
         -0.05684932, -0.17854682, -0.50577176,  0.4026775 , -0.42308894],
        [-0.8405634 , -0.66435444, -0.05299184,  0.29900095,  0.13477863,
          0.44797152, -0.76723933,  0.10772793,  0.4661794 , -0.25249785],
        [ 0.73105717,  0.79097736, -0.2864933 , -0.0170467

In [44]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_rmsprop_2nd.h5')